In [2]:
import streamlit as st
import json
import geopandas as gpd
import pyproj
import plotly.graph_objs as go

In [3]:
polygon = gpd.read_file(r"cb_2021_12_bg_500k.shp")

In [4]:
polygon.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,AFFGEOID,GEOID,NAME,NAMELSAD,LSAD,ALAND,AWATER,geometry
0,12,086,004404,1,1500000US120860044041,120860044041,1,Block Group 1,BG,240075,0,"POLYGON ((-80.13092 25.77916, -80.13143 25.779..."
1,12,011,031203,1,1500000US120110312031,120110312031,1,Block Group 1,BG,308482,977442,"POLYGON ((-80.09032 26.23688, -80.08846 26.239..."
2,12,009,066400,4,1500000US120090664004,120090664004,4,Block Group 4,BG,509762,416388,"POLYGON ((-80.58768 28.14790, -80.58701 28.149..."
3,12,103,026001,2,1500000US121030260012,121030260012,2,Block Group 2,BG,516570,1821258,"POLYGON ((-82.81812 27.98631, -82.81735 27.985..."
4,12,086,001801,2,1500000US120860018012,120860018012,2,Block Group 2,BG,329987,0,"POLYGON ((-80.23286 25.82782, -80.22877 25.827..."


In [5]:
# project GeoPandas dataframe
map_df = polygon 
map_df.to_crs(pyproj.CRS.from_epsg(4326), inplace=True)

In [6]:
# # reading in the points shapefile
# points = gpd.read_file(r".shp")
# # project GeoPandas dataframe
# points.to_crs(pyproj.CRS.from_epsg(4326), inplace=True)
# # define lat, long for points
# Lat = points['Lat']
# Long = points['Long']

In [7]:
import os
import pathlib
import requests
import zipfile
import geopandas as gpd
import streamlit as st
import leafmap.colormaps as cm
from leafmap.common import hex_to_rgb


STREAMLIT_STATIC_PATH = pathlib.Path(st.__path__[0]) / "static"
# We create a downloads directory within the streamlit static asset directory
# and we write output files to it
DOWNLOADS_PATH = STREAMLIT_STATIC_PATH / "downloads"
if not DOWNLOADS_PATH.is_dir():
    DOWNLOADS_PATH.mkdir()

def get_geom_data(category):

    prefix = (
        ""
    )
    links = {
        "national": prefix + "us_nation.geojson",
        "state": prefix + "us_states.geojson",
        "county": prefix + "us_counties.geojson",
        "metro": prefix + "us_metro_areas.geojson",
        "zip": "https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_zcta510_500k.zip",
    }

    if category.lower() == "zip":
        r = requests.get(links[category])
        out_zip = os.path.join(DOWNLOADS_PATH, "cb_2018_us_zcta510_500k.zip")
        with open(out_zip, "wb") as code:
            code.write(r.content)
        zip_ref = zipfile.ZipFile(out_zip, "r")
        zip_ref.extractall(DOWNLOADS_PATH)
        gdf = gpd.read_file(out_zip.replace("zip", "shp"))
    else:
        gdf = gpd.read_file(links[category])
    return gdf


def join_attributes(gdf, df, category):

    new_gdf = None
    if category == "county":
        new_gdf = gdf.merge(df, left_on="GEOID", right_on="county_fips", how="outer")
    elif category == "state":
        new_gdf = gdf.merge(df, left_on="STUSPS", right_on="STUSPS", how="outer")
    elif category == "national":
        if "geo_country" in df.columns.values.tolist():
            df["country"] = None
            df.loc[0, "country"] = "United States"
        new_gdf = gdf.merge(df, left_on="NAME", right_on="country", how="outer")
    elif category == "metro":
        new_gdf = gdf.merge(df, left_on="CBSAFP", right_on="cbsa_code", how="outer")
    elif category == "zip":
        new_gdf = gdf.merge(df, left_on="GEOID10", right_on="postal_code", how="outer")
    return new_gdf


In [9]:
# set GeoJSON file path
path = r"C:\Users\smyu2\OneDrive\GitHub\CountyAssessmentProj\CountyAssessmentProject\mainProj\us_states.geojson"
# write GeoJSON to file
map_df.to_file(path, driver = "GeoJSON")
with open(path) as geofile:
   j_file = json.load(geofile)
# index geojson
i=1
for feature in j_file["features"]:
   feature ['id'] = str(i).zfill(2)
   i += 1

In [14]:
# mapbox token
mapboxt = 'MapBox Token'
 
# define layers and plot map
choro = go.Choroplethmapbox(z=map_df['STFIPS'], locations =  
        map_df.index, colorscale = 'Viridis', geojson = j_file, 
        text = map_df['NAME'], marker_line_width=0.1) 
scatt = go.Scattermapbox(lat=Lat, lon=Long,mode='markers+text',    
        below='False', marker=dict( size=12, color ='rgb(56, 44, 100)'))
layout = go.Layout(title_text ='USA Cities', title_x =0.5,  
         width=950, height=700,mapbox = dict(center= dict(lat=37,  
         lon=-95),accesstoken= mapboxt, zoom=4,style="stamen-terrain"))

KeyError: 'STFIPS'

In [ ]:
# streamlit multiselect widget
layer1 = st.multiselect('Layer Selection', [choro, scatt], 
         format_func=lambda x: 'Polygon' if x==choro else 'Points')
# assign Graph Objects figure
fig = go.Figure(data=layer1, layout=layout)
# display streamlit map
st.plotly_chart(fig)